Il metodo scelto per l'assignment 1 è stato il metodo di Zhang [(Link alla pagina wikipedia)](https://en.wikipedia.org/wiki/Camera_resectioning#:~:text=Zhang%27s%20method%5Bedit%5D) 

I passaggi da effettuare sono
- Corrispondenza punti 2D e 3D
- Trovare la matrice P
- Trovare K che è la matrice R della fattorizzazione precedente
- Mentre la matrice R di rotazione (da non confondere con la R di RQ) sarà la matrice Q di RQ
- Trovare t vettore colonna della traslazione

In [ ]:
import Calibrate as cb
import numpy as np
import cv2

Chiamo la funzione per calibrare la videocamera con 4 foto

In [ ]:
CameraIntrinsic4, CameraIntrinsicCalcolataDaCV24, Errore4 = cb.calibra(4)

print("K calcolata: \n", CameraIntrinsic4)
print("CalibrateCamera: \n", CameraIntrinsicCalcolataDaCV24)
print("Errore: \n",Errore4)

Chiamo la funzione per calibrare la videocamera con 8 foto

In [ ]:
CameraIntrinsic8, CameraIntrinsicCalcolataDaCV28, Errore8 = cb.calibra(8)

print("K calcolata: \n", CameraIntrinsic8)
print("CalibrateCamera: \n", CameraIntrinsicCalcolataDaCV28)
print("Errore: \n",Errore8)

Chiamo la funzione per calibrare la videocamera con 16 foto

In [ ]:
CameraIntrinsic16, CameraIntrinsicCalcolataDaCV216, Errore16 = cb.calibra(16)

print("K calcolata: \n", CameraIntrinsic16)
print("CalibrateCamera: \n", CameraIntrinsicCalcolataDaCV216)
print("Errore: \n",Errore16)

Creo le corrispondenze 2d/3d dei punti nel pavimento del laboratorio

In [ ]:
dueD = np.array([[496,605], [659,341], [481,564], [482,649], [511,560], [512,649] ])
treD = np.array([[0,0,0], [-132,-96], [-14.75,21] , [14.75,21], [-14.75,-21], [14.75,-21] ])

Definisco un doppio ciclo for, il primo serve per ciclare in un range 0,3 estremi inclusi per dividere il set di immagini in 4 per fare cross validation, il set di validazione sarà dato dal vettore tagliato dalla posizione iesima alla i+1 esima



In [ ]:
for i in range (0,4):
    multiply = int(len(immagini)/4)
    a = int(multiply * i)
    b = int(multiply * i +1)
    validation = immagini[a : b]
    training = [i for i in immagini if i not in validation]
    for im in training:
        image = cv2.imread(im)
        scalagrigi = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        ret, angoli = cv2.findChessboardCorners(scalagrigi, scacchiera, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

        if ret:
            punti3d.append(oggetto3d)
            angoli2d = cv2.cornerSubPix(scalagrigi, angoli, (11,11),(-1,-1), criteri)
            punti2d.append(angoli2d)
            #cal = calibra()
            #errori[i] = cv2.calibrateCamera() - cal



        cv2.imshow('immagine', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
 

Disegno quindi i punti a terra data l'immagine, la matrice di rotazione, di traslazione e 

In [ ]:
def draw_ground(img, R, T, K):
    xg = np.arange(-5, 10, 0.5) 
    yg = np.arange(-5, 10, 0.5) 
    xx, yy = np.meshgrid(xg, yg)

    dim = xx.shape[0]*xx.shape[1]
    points = np.zeros((dim,3), np.float32)

    xx = xx.reshape(dim)
    yy = yy.reshape(dim)

    points[:,0] = xx 
    points[:,1] = yy 
    points[:,2] = np.zeros((dim))

    ground, _ = cv2.projectPoints(points, R, T, K)
    ground = np.squeeze(ground).astype(np.int32)

    img_to_show_res = img.copy()
    for p in ground:
        img_to_show_res = cv2.circle(img_to_show_res, p, 3, (255, 0, 0) )

    cv2.imshow("ground", img_to_show_res)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    


I punti rilevati in laboratorio si trovano alle coordinate espresse in cm per lo spazio 3d e pixel per lo spazio 2d:
| Immagine | X3D  | Y3D  | X2D | Y2D |
| -------- | ---- | ---- | --- | --- |
| 125956   | -200 | 100  | 774 | 338 |
| 13411    | -200 | 0    | 621 | 336 |
| 13444    | -200 | -100 | 470 | 332 |
| 13519    | -100 | -100 | 438 | 400 |
| 13553    | -100 | 0    | 614 | 402 |
| 13631    | -100 | 100  | 790 | 404 |
| 1370     | 0    | -100 | 396 | 485 |
| 13746    | 0    | 0    | 607 | 492 |
| 13815    | 0    | 100  | 817 | 495 |
| 13845    | 100  | -100 | 340 | 615 |
| 13913    | 100  | 0    | 595 | 622 |
| 13947    | 100  | 100  | 858 | 628 |
| 131522   | 100  | -200 | 120 | 594 |
| 131546   | 0    | -200 | 208 | 473 |
| 13163    | -100 | -200 | 276 | 393 |
| 131647   | -300 | 0    | 626 | 288 |


In [ ]:
_3d = np.array([ [-200,100], [-200,0], [-200,-100], [-100,-100], [-100,0], [-100,100] ,[0,-100], [0,0], [0,100], [100,-100], [100,0], [100,100], [100,-200], [0,-200], [-100,-200], [-300,0]])
_2d = np.array([ [774,338], [621,336], [470,332], [438,400], [614,402], [790,404], [396,485], [607,492], [817,495], [340,615], [595,622], [858,628], [120,594], [208,473], [276,393], [626,288]])


   